In [1]:
from utils import model_tools, db_tools, data_processing
import time

# 设置Chat Model和Code Model
model_tools.current_chat_model = 'gpt-3.5-turbo'
model_tools.current_code_model = 'ZhipuAI/codegeex4-all-9b'
#model_tools.load_model('codegeex2-6b')
model_tools.load_model('ZhipuAI/codegeex4-all-9b')

######## 需要人工设置 ########
data_file = './data/数据表字段说明-精简1.txt'  # 字段定义
qa_file = './qa_list-2.txt'                  # QA测试题
sql_file = './data/create_sql.txt'           # SQL数据表

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
# 得到sql
def get_sql(query):
    prompt = f"""-- language: SQL
### Question: {query}
### Input: {create_sql}
### Response:
Here is the SQL query I have generated to answer the question `{query}`:
```sql
"""

    # 调用CODE模型
    #response = model_tools.model_run(tokenizer, model, prompt, max_length=len(prompt)+200)
    #response = model_tools.model_run(tokenizer, model, prompt, max_length=len(prompt))
    response = model_tools.model_code(prompt, max_length=len(prompt), top_k=1, temperature=0.01)
    content = response['content']
    return content, len(prompt)

# 读取 SQL数据表
with open(sql_file, 'r', encoding='utf-8') as file:
    create_sql = file.read()
# 读取 数据表字段说明-精简
with open(data_file, 'r', encoding='utf-8') as file:
    table_description = file.read()
# 读取 SQL问题列表
with open(qa_file, 'r', encoding='utf-8') as file:
    qa_list = file.read()
qa_list = qa_list.split('=====')

# 保存SQL结果
sql_list = []
markdown_list = []
time_list = []
for qa in qa_list:
    query = qa
    query = query.replace('\n', '')
    #print(query)
    start_time = time.time()
    # 请求生成sql
    content, prompt_len = get_sql(query)
    use_time = round(time.time()-start_time, 2)
    time_list.append(use_time)
    print('SQL生成时间：', use_time)
    print('prompt_len=', prompt_len)
    print('content=', content)
    # 提取生成的SQL
    sql = data_processing.get_sql_code(content)
    print('SQL: {}'.format(sql))
    sql_list.append(sql)

用时: 2.2 Token使用：690
SQL生成时间： 2.2
prompt_len= 2888
content= [gMASK] <sop> -- language: SQL
### Question: 获取所有客户的姓名和联系电话。
### Input: CREATE TABLE CustomerInfo (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(50),
    Gender VARCHAR(10),
    DateOfBirth DATE,
    IDNumber VARCHAR(18),
    Address VARCHAR(100),
    PhoneNumber VARCHAR(20),
    EmailAddress VARCHAR(50),
    MaritalStatus VARCHAR(20),
    Occupation VARCHAR(50),
    HealthStatus VARCHAR(20),
    RegistrationDate DATE,
    CustomerType VARCHAR(20),
    SourceOfCustomer VARCHAR(50),
    CustomerStatus VARCHAR(20)
);

CREATE TABLE PolicyInfo (
    PolicyNumber INT PRIMARY KEY,
    CustomerID INT,
    ProductID INT,
    PolicyStatus VARCHAR(20),
    Beneficiary VARCHAR(50),
    Relationship VARCHAR(20),
    PolicyStartDate DATE,
    PolicyEndDate DATE,
    PremiumPaymentStatus VARCHAR(20),
    PaymentDate DATE,
    PaymentMethod VARCHAR(20),
    AgentID INT
);

CREATE TABLE ClaimInfo (
    ClaimNumber INT PRIMARY KEY,
    Poli

In [4]:
#print(response)
#response[prompt_len:]

In [3]:
time_list

[2.2, 0.82, 1.49, 2.03, 0.76, 0.61, 0.62, 1.02, 1.1, 1.02]

In [4]:
import pandas as pd

result = pd.DataFrame(columns=['QA', 'SQL', 'time'])
result['QA'] = qa_list
result['SQL'] = sql_list
result['time'] = time_list
result.to_excel('./sql_result_codegeex-1.xlsx', index=False)
result

,QA,SQL,time
0,获取所有客户的姓名和联系电话。\n,"SELECT Name, PhoneNumber FROM CustomerInfo;\n",2.20
1,\n找出所有已婚客户的客户ID和配偶姓名。\n,"SELECT CustomerID, SpouseName\nFROM CustomerIn...",0.82
2,\n查询所有未支付保费的保单号和客户姓名。\n,"SELECT PolicyInfo.PolicyNumber, CustomerInfo.N...",1.49
3,\n找出所有理赔金额大于10000元的理赔记录，并列出相关客户的姓名和联系电话。\n,"SELECT c.Name, c.PhoneNumber\nFROM ClaimInfo A...",2.03
4,\n查找代理人的姓名和执照到期日期，按照执照到期日期升序排序。\n,"SELECT Name, LicenseExpirationDate\nFROM Agent...",0.76
5,\n获取所有保险产品的产品名称和保费，按照保费降序排序。\n,"SELECT ProductName, Premium\nFROM ProductInfo\...",0.61
6,\n查询销售区域在上海的员工，列出他们的姓名和职位。\n,"SELECT Name, Position\nFROM EmployeeInfo\nWHER...",0.62
7,\n找出所有年龄在30岁以下的客户，并列出其客户ID、姓名和出生日期。\n,"SELECT CustomerID, Name, DateOfBirth\nFROM Cus...",1.02
8,\n查找所有已审核但尚未支付的理赔记录，包括理赔号、审核人和审核日期。\n,"SELECT ClaimNumber, ClaimHandler, ReviewDate\n...",1.10
9,\n获取每个产品类型下的平均保费，以及该产品类型下的产品数量。,"SELECT ProductType, AVG(Premium) AS AveragePre...",1.02
